In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models,layers
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [3]:
data_fashion=fashion_mnist

In [4]:
(X_train, y_train), (X_test, y_test) = data_fashion.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0
X_train

class_names=['T shirt','Trouser','Pullover','dress','Coat','Sandal',
             'Shirt','Sneaker','Bag','Ankle boot']

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
X_train.shape

(60000, 28, 28)

In [8]:
y_train.shape

(60000,)

In [6]:
model = Sequential([
    layers.Flatten(input_shape=(28, 28)),     # Flatten the 28x28 images
    layers.Dense(units=128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(units=60,activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(units=10, activation='softmax')
])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 60)             │         7,740 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 60)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 108,830 (425.12 KB)

 Trainable params: 108,830 (425.12 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

In [8]:
from re import VERBOSE
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_fashion_model.keras', save_best_only=True,
                                   monitor='val_loss',
                                   mode='min',
                                   verbose=1)


In [10]:
overall_hist=model.fit(X_train,
                       y_train,
                       epochs=50,
                       validation_data=(X_test,y_test),
                       callbacks=[early_stopping,model_checkpoint],
                       verbose=1)

Epoch 1/50
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7118 - loss: 0.8117
Epoch 1: val_loss improved from inf to 0.45691, saving model to best_fashion_model.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7119 - loss: 0.8116 - val_accuracy: 0.8351 - val_loss: 0.4569
Epoch 2/50
1865/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8352 - loss: 0.4625
Epoch 2: val_loss improved from 0.45691 to 0.40992, saving model to best_fashion_model.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.8352 - loss: 0.4624 - val_accuracy: 0.8514 - val_loss: 0.4099
Epoch 3/50
1866/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8499 - loss: 0.4172
Epoch 3: val_loss improved from 0.40992 to 0.39381, saving model to best_fashion_model.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.8500 - loss: 0.4172 - val_accuracy: 0.8519 - val_loss: 0.3938
Epoch 4/50
1872/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8545 - loss: 0.3983
Epoch 4: val_

In [23]:
from tensorflow.keras.models import load_model

best_model = load_model('best_fashion_model.keras')
y_pred_probs = best_model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [24]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))

# Confusion Matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)


Classification Report:
              precision    recall  f1-score   support

     T shirt       0.82      0.85      0.84      1000
     Trouser       0.99      0.96      0.98      1000
    Pullover       0.74      0.81      0.78      1000
       dress       0.87      0.89      0.88      1000
        Coat       0.75      0.83      0.79      1000
      Sandal       0.98      0.95      0.96      1000
       Shirt       0.78      0.58      0.67      1000
     Sneaker       0.93      0.96      0.95      1000
         Bag       0.96      0.98      0.97      1000
  Ankle boot       0.95      0.96      0.95      1000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000


Confusion Matrix:
[[854   0  29  35   4   0  68   0  10   0]
 [  1 963   3  24   5   0   3   0   1   0]
 [ 12   0 815   9 127   0  31   0   6   0]
 [ 28   4  15 893  40   0  16   0   4   0]
 [  0   0 104  25 830   0 